<a href="https://colab.research.google.com/github/prab-tri/demo/blob/master/ICP06_Big_data_Apps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ICP06 - Big Data and Analytics 
Task - ICP- 6 Use a different data and use the model provided in ICP6 to perform Text generation. You must make 4 changes (for example adding more layers to model, changing hyperparameters etc ) in the source code. Report your findings in detail. Note: please indicate in your reports which 4 changes you made in the source code and why in your opinion these changes are logical.

In [1]:
#Import all important libraries
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
#uploading the file from local drive 
from google.colab import files
uploaded = files.upload()

Saving sherlock_homes.txt to sherlock_homes.txt


Read the data

In [3]:
# Read, then decode for py2 compat.
text = open('sherlock_homes.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 581880 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Proje


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

98 unique characters


In [6]:
#view first 10 characters 
vocab[0:10]

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(']

98 unique characters

Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
#converting text to integers using mapping
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
#view of text as integers
text_as_int[0:10]

array([45, 76, 73, 68, 63, 61, 78,  1, 36, 79])

The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

P
r
o
j
e
c
t
 
G
u


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle\n\nThis eBook is for the u"
'se of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give i'
't away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or onli'
'ne at www.gutenberg.net\n\n\nTitle: The Adventures of Sherlock Holmes\n\nAuthor: Arthur Conan Doyle\n\nRelea'
'se Date: November 29, 2002 [EBook #1661]\nLast Updated: May 20, 2019\n\nLanguage: English\n\nCharacter set'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [11]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle\n\nThis eBook is for the "
Target data: "roject Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle\n\nThis eBook is for the u"


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [13]:
# Batch size ##1st Change in parameter 
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
#Drop out rate
drop_rate = 0.1

In [17]:
#Second change - Adding activation layer as tanh in GRU
#Third change  - Adding kernel initializer as glorot_uniform to gru layer
#fourth change - Adding dropout layer with 0.1 drop rate
#Adding activation function to dense layer  
def build_model(vocab_size, embedding_dim, rnn_units, batch_size, drop_rate):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        activation='tanh',
                        return_sequences=True,
                        kernel_initializer='glorot_uniform',
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(drop_rate),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    drop_rate = drop_rate)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 98) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          25088     
_________________________________________________________________
gru (GRU)                    (128, None, 1024)         3938304   
_________________________________________________________________
dropout (Dropout)            (128, None, 1024)         0         
_________________________________________________________________
dense (Dense)                (128, None, 98)           100450    
Total params: 4,063,842
Trainable params: 4,063,842
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([62, 54, 27, 52, 82, 72,  8, 80, 42, 21, 13, 33, 16,  1, 31, 29, 78,
        4,  9, 75, 30, 29, 27, 49, 11, 62, 83, 49, 80, 90, 70, 31, 82, 50,
       27, 83, 56, 31, 89, 40, 50, 67, 36,  3, 82, 93, 35, 62, 59, 79, 15,
       81, 91, 20, 38, 94, 22, 45, 61, 84, 38,  0, 72, 14, 43, 50, 76, 53,
       89, 66, 64,  6, 39, 35,  6, 84, 75, 63, 45, 96, 41, 58,  7, 62, 61,
       12, 49, 45,  8,  2, 54, 23, 41, 74, 76, 93, 71, 17, 59, 76])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 100, 98)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5849614


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [23]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 10 epochs to train the model.

In [25]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
45/45 [==============================] - 376s 8s/step - loss: 3.5668
Epoch 2/10
45/45 [==============================] - 376s 8s/step - loss: 2.5149
Epoch 3/10
45/45 [==============================] - 381s 8s/step - loss: 2.3064
Epoch 4/10
45/45 [==============================] - 375s 8s/step - loss: 2.1862
Epoch 5/10
45/45 [==============================] - 372s 8s/step - loss: 2.0605
Epoch 6/10
45/45 [==============================] - 378s 8s/step - loss: 1.9465
Epoch 7/10
45/45 [==============================] - 374s 8s/step - loss: 1.8490
Epoch 8/10
45/45 [==============================] - 376s 8s/step - loss: 1.7603
Epoch 9/10
45/45 [==============================] - 375s 8s/step - loss: 1.6793
Epoch 10/10
45/45 [==============================] - 372s 8s/step - loss: 1.6087


Generate text

Restore the latest checkpoint To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1, drop_rate=drop_rate)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            25088     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dropout_2 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
dense_2 (Dense)              (1, None, 98)             100450    
Total params: 4,063,842
Trainable params: 4,063,842
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [30]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [31]:
print(generate_text(model, start_string=u"Sherlock "))

Sherlock Holmes, sabling in. I have gooming engaination be dasileds.”

“Nest had been rount cosc, to the fronathlow. Sirnumaswer Street. “The enversed from this shottle redlidnding upon
his pacted up with whe has fireccuents. As time, all which had jought out itto the first, their
of it, sincastecfid the
ladgU goven?”


He tases.
I beer tame a meghind one of the pockistes besses im agoin normiss and into the donething with mught that sellow leme
undated that I those exfuthed, for deating-what ot faighter atvertuge.

I care, Mr. Rucast my been someming had her findstand without if have
been out room door unow
mantered over the trubbly pay wret atsle to the house. Nomembark So Sime with
Steenatient. You what I wonly my, and his shell wall of agress—to whe had the whene gave we speak om whe was the police dow. Home was the mory bate to to stand your
fearly collacy, gazed younghen, is sutsing of son, the sborence from caused to be on. The bad happered croming lit be the thr last conloction